In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator    
## load_img(a.jpg) --> 이미지를 메모리에 로드
## img_to_array --> 메모리에 로드된 이미지의 픽셀값을 배열화
## ImageDataGenerator --> 메모리에 로드된 이미지를 변형해 또 다른 이미지로 "증강", 증강시에는 데이터프레임화
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, concatenate, Input, Flatten, Dense, MaxPool2D
from keras.datasets import  mnist

import os
import warnings
warnings.filterwarnings(action="ignore")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # tensorflow오류
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

np.random.seed(121)
tf.random.set_seed(121)

In [3]:
def CHART_PLOT_HISTORY(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(8,12))

    plt.subplot(2,1,1)
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.plot(hist['epoch'], hist['accuracy'],    label = 'acc')
    plt.plot(hist['epoch'], hist['val_accuracy'],label = 'val_acc')
    plt.legend()

    plt.subplot(2,1,2)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.plot(hist['epoch'], hist['loss'],    label = 'loss')
    plt.plot(hist['epoch'], hist['val_loss'],label = 'val_loss')
    plt.legend()
    plt.show()

In [ ]:
d_path = "/content/drive/MyDrive/Colab Notebooks/pkg/dl_exam/cat-dog/cat-dog1/"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,    # 소수점 살리기
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
flow_train_img = train_datagen.flow_from_directory(
    directory = d_path + "train",
    target_size = (128,128),
    batch_size = 32
)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1.0/255)

In [ ]:
flow_test_img = test_datagen.flow_from_directory(
    directory = d_path + "test",
    target_size = (128,128),
    batch_size = 32
)

### 이부분 확인

In [ ]:
flow_val_img = train_datagen.flow_from_directory(
    directory = d_path + "train",
    target_size = (128,128),
    batch_size = 32
)

In [ ]:
model = Sequential()
                                                                                                # 128, 128, 3
model.add(Conv2D(filters=20, kernel_size=(5,5), input_shape=(128,128,3), activation='relu'))    # 124, 124, 20
model.add(MaxPool2D(pool_size=(2, 2)))                                                          # 62, 62, 20 
model.add(BatchNormalization())

model.add(Conv2D(filters=40, kernel_size=(3,3), activation='relu'))                             # 60, 60, 40
model.add(MaxPool2D(pool_size=(2, 2)))                                                          # 30, 30, 40
model.add(BatchNormalization())

model.add(Conv2D(filters=60, kernel_size=(3,3), activation='relu'))                             # 28 28 60
model.add(MaxPool2D(pool_size=(2, 2)))                                                          # 14 14 60
model.add(BatchNormalization())

model.add(Conv2D(filters=80, kernel_size=(3,3), activation='relu'))                             # 12 12 80
model.add(MaxPool2D(pool_size=(2, 2)))                                                          # 6 6 80
model.add(BatchNormalization())

model.add(Flatten())                                
model.add(Dense(64, activation='relu')) 
model.add(Dense(2, activation='softmax'))
print(model.summary())  

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=10)

In [ ]:
model.compile(optimizer="adam",   # GD(경사하강) SGD momentum ada.. adam
              loss='binary_crossentropy', # 그때그때 달라지는 부분!! 확인필요
              metrics=["accuracy"])

In [ ]:
history = model.fit(dg_train_img,   #X_train, y_train
                    steps_per_epoch=np.ceil(flow_val_img.samples/100),  #batch_size=17500/350=150
                    epochs=3,
                    callbacks=[early_stop],
                    validation_data=(flow_val_img), #validation_data=(X_test, y_test)
                    validation_steps=np.ceil(flow_val_img.samples/100)
                    )